<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/142-Langchain_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain이란?
* Language : Large Language Model의 Language
* Chain: 모듈들을 체인으로 연결하여 다른 툴의 출력을 다음 툴의 입력으로 연결하는 방식
  * 예: 검색 -> OpenAI(LLM) -> 계산
* LLM(Large Language Model)을 이용해서 뭔가 만들때 쓸수 있는 툴/라이브러리 모음.
* 파이썬과 자바스크립트 라이브러리를 제공.
* 검색, 메모리, 계산, 요약, 색인, Agent(에이전트) 등의 기능을 제공
* 모듈화로 쉽게 부품을 갈아끼울수 있음
* LLM은 기본 OpenAI (text-davinci-003) 이용

---
참고 자료:
* 파이썬 문서: https://python.langchain.com/en/latest/index.html
* 개념 문서: https://docs.langchain.com/docs/
* 소스: https://github.com/hwchase17/langchain

---

In [ ]:
#@title 기본 패키지(openai, langchain) 설치
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00


In [ ]:
#@title 기타 패키지 설치 (구글검색, 위키피디아, VectorStore, HuggingFace Embedding)
!pip install google-search-results
!pip install wikipedia
!pip install faiss-cpu # 오픈소스 벡터DB (Facebook, MIT license)
!pip install sentence_transformers # HuggingFace Embedding 사용 위해서 필요
!pip install tiktoken # Summarization 할때 필요

In [ ]:
#@title 0. API 키 설정
import os
#@markdown https://platform.openai.com/account/api-keys
OPENAI_API_KEY = "sk-" #@param {type:"string"}

#@markdown https://huggingface.co/settings/tokens
#@markdown HuggingFace에서 모델 다운로드나 클라우드 모델 사용하기 위해서 필요 (무료)
HUGGINGFACEHUB_API_TOKEN = "hf_" #@param {type:"string"}

#@markdown https://serpapi.com/manage-api-key
#@markdown 구글 검색하기 위해서 필요 (월 100회 무료)
SERPAPI_API_KEY = "" #@param {type:"string"}

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY


In [ ]:
#@title 1. OpenAI LLM (text-davinci-003)

from langchain.llms import OpenAI

In [ ]:
#@title 1. OpenAI LLM (text-davinci-003)

from langchain.llms import OpenAI

llm = OpenAI(model_name='text-davinci-003', temperature=0.9)



In [ ]:
type(llm)

langchain.llms.openai.OpenAI

In [ ]:
llm('미국의 빌보드차트에 오른 한국 가수 이름을 알려 주세요.')

"\n\n1. PSY - Gangnam Style\n2. BTS - DNA\n3. BLACKPINK - Kill This Love\n4. EXO - Growl\n5. TWICE - Chaser\n6. BIGBANG - Bang Bang Bang\n7. Taeyang - Wedding Dress\n8. IU - Palette\n9. SEVENTEEN - Don't Wanna Cry\n10. MONSTA X - Beautiful"

In [ ]:
llm('2020 올림픽은 어디에서 개최 되었나요?')

'\n\n2020년 올림픽은 러시아 모스크바에서 개최되었습니다.'

In [ ]:
#@title 2. ChatOpenAI LLM (gpt-3.5-turbo)

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
sys = SystemMessage(content="당신은 스포츠 전문 AI입니다.")
msg = HumanMessage(content='2020 하계 올림픽에서 대한민국 금메달 획득 갯수는?')

In [ ]:
aimsg = chat([sys, msg])
aimsg.content

'2020 하계 올림픽에서 대한민국은 총 6개의 금메달을 획득했습니다.'

In [ ]:
#@title 3. Prompt Template & chain

from langchain.prompts import PromptTemplate

In [ ]:

prompt = PromptTemplate(
    input_variables=["상품"],
    template="{상품} 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로",
)



In [ ]:
prompt.format(상품="신발")

'신발 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로'

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=prompt)

# chain.run("AI 여행 추천 서비스")
chain.run(상품="콜라")

'1. 청하금산음료\n2. 복숭이콜라\n3. 불멸의콜라\n4. 향긋한콜라\n5. 순수한맛콜라\n6. 한가득콜라\n7. 금강산콜라\n8. 화려한맛콜라\n9. 푸른하늘콜라\n10. 미소가득콜라'

In [ ]:
#@title 4. ChatPromptTemplate & chain

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
chat = ChatOpenAI(temperature=0)

template="You are a helpful assisstant that tranlates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


chatchain = LLMChain(llm=chat, prompt=chat_prompt)
chatchain.run(input_language="English", output_language="Korean", text="Thanks god TGIF!")


'감사하다 신이여, 주말이다!'

In [ ]:
#@title 5. Agents and Tools

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

* Tool: 특정 일을 하는 역할. 예) 구글 검색, 디비 조회, Python 실행/계산.
* Agents: 어떤 툴을 어떤 순서로 실행할지 결정하는 역할 (LLM 이용)

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm=chat)
# tools = load_tools(["wikipedia", "llm-math"], llm=chat)

agent = initialize_agent(tools, llm=chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("한국의 'New Jeans' 그룹 멤버의 인원 수는? 오늘 2023년 11월 17일 현재 전체 멤버들의 나이의 합은?") # 한국의 'New Jeans' 그룹 멤버의 인원 수는? 오늘 2023년 5월 30일 현재 전체 멤버들의 나이의 합은 얼마인가요



> Entering new AgentExecutor chain...
I need to find information about the number of members in the Korean group 'New Jeans' and their ages.
Action: Search
Action Input: "New Jeans Korean group members"
Observation: The group is composed of five members: Minji, Hanni, Danielle, Haerin, and Hyein. NewJeans is known for their girl next door image and 1990s- and 2000s-indebted pop and R&B songs with influences of various dance and club styles.
Thought:I still need to find information about the ages of the members.
Action: Search
Action Input: "New Jeans Korean group members ages"
Observation: Minji was born on May 7, 2004 and is 19 years old. Hanni was born on October 6, 2004 and is 18 years old. Danielle was born on April 11, 2005 and is 18 years old. Haerin was born on May 15, 2006 and is 17 years old.
Thought:I have all the information I need to calculate the sum of the ages.
Action: Calculator
Action Input: 19 + 18 + 18 + 17
Observation: Answer: 72
Thought:I now know the final answe

"The sum of the ages of the members of the Korean group 'New Jeans' as of June 23, 2023 is 72."

In [ ]:
len(agent.tools)

2

In [ ]:
print(agent.tools[0].description)
print(agent.tools[1].description)

A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Useful for when you need to answer questions about math.


In [ ]:
#@title 6. Memory
from langchain import ConversationChain

conversation = ConversationChain(llm=chat, verbose=True)
conversation.predict(input="인공지능에서 Transformer가 뭐야?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 인공지능에서 Transformer가 뭐야?
AI:

> Finished chain.


'Transformer는 자연어 처리 분야에서 매우 중요한 모델입니다. 이 모델은 주로 기계 번역, 질의응답, 요약 등의 작업에 사용됩니다. Transformer는 2017년에 발표된 논문 "Attention is All You Need"에서 처음 소개되었습니다. 이 모델은 기존의 순차적인 방식이 아닌, 전체 문장을 동시에 처리하는 방식을 사용하여 성능을 크게 향상시켰습니다. Transformer는 주로 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이를 통해 문장의 의미를 잘 이해하고 번역, 요약 등의 작업을 수행할 수 있습니다.'

In [ ]:
conversation.predict(input="RNN하고 차이 설명해줘.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 인공지능에서 Transformer가 뭐야?
AI: Transformer는 자연어 처리 분야에서 매우 중요한 모델입니다. 이 모델은 주로 기계 번역, 질의응답, 요약 등의 작업에 사용됩니다. Transformer는 2017년에 발표된 논문 "Attention is All You Need"에서 처음 소개되었습니다. 이 모델은 기존의 순차적인 방식이 아닌, 전체 문장을 동시에 처리하는 방식을 사용하여 성능을 크게 향상시켰습니다. Transformer는 주로 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이를 통해 문장의 의미를 잘 이해하고 번역, 요약 등의 작업을 수행할 수 있습니다.
Human: RNN하고 차이 설명해줘.
AI:

> Finished chain.


'RNN은 순환 신경망(Recurrent Neural Network)의 약자로, 순차적인 데이터를 처리하는 데에 주로 사용되는 모델입니다. RNN은 이전의 입력에 의존하여 현재의 출력을 계산하는 구조를 가지고 있습니다. 이전의 정보를 기억하고 현재의 입력과 함께 사용하여 다음 출력을 예측하는 데에 유용합니다. 예를 들어, 문장의 단어를 순서대로 입력으로 받아 문장의 의미를 파악하거나, 시계열 데이터를 입력으로 받아 다음 시점의 값을 예측하는 등의 작업에 사용됩니다.\n\n반면에 Transformer는 RNN과는 다른 구조를 가지고 있습니다. Transformer는 기존의 순차적인 방식이 아닌, 전체 문장을 동시에 처리하는 방식을 사용합니다. 이를 위해 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이러한 구조를 통해 Transformer는 RNN보다 더 긴 문장을 처리하는 데에 효과적이며, 번역, 요약 등의 작업에서 뛰어난 성능을 보입니다.'

In [ ]:
conversation.predict(input="Seq2Seq 와도 차이점을 비교해줘.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 인공지능에서 Transformer가 뭐야?
AI: Transformer는 자연어 처리 분야에서 매우 중요한 모델입니다. 이 모델은 주로 기계 번역, 질의응답, 요약 등의 작업에 사용됩니다. Transformer는 2017년에 발표된 논문 "Attention is All You Need"에서 처음 소개되었습니다. 이 모델은 기존의 순차적인 방식이 아닌, 전체 문장을 동시에 처리하는 방식을 사용하여 성능을 크게 향상시켰습니다. Transformer는 주로 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이를 통해 문장의 의미를 잘 이해하고 번역, 요약 등의 작업을 수행할 수 있습니다.
Human: RNN하고 차이 설명해줘.
AI: RNN은 순환 신경망(Recurrent Neural Network)의 약자로, 순차적인 데이터를 처리하는 데에 주로 사용되는 모델입니다. RNN은 이전의 입력에 의존하여 현재의 출력을 계산하는 구조를 가지고 있습니다. 이전의 정보를 기억하고 현재의 입력과 함께 사용하여 다음 출력을 예측하는 데에 유용합니다. 예를 들어, 문장의 단어를 순서대로 입력으로 받아 문장의 의미를 파악하거나, 시계열 데이터를 입력으로 받아 다음 시점의 값을 예측하는 등의 작업에 사용됩니다.

반면에 Transformer는 RNN과는 다른 구조를 

'Seq2Seq는 Sequence-to-Sequence의 약자로, 번역, 대화 시스템 등에서 사용되는 모델입니다. Seq2Seq 모델은 주로 RNN을 기반으로 구성되며, 입력 시퀀스를 고정된 길이의 벡터로 인코딩한 후, 이 벡터를 기반으로 출력 시퀀스를 생성합니다. 이 모델은 입력 시퀀스와 출력 시퀀스의 길이가 다를 수 있으며, 입력과 출력 사이의 관계를 학습하여 번역이나 대화를 수행합니다.\n\nTransformer는 Seq2Seq 모델과는 다른 구조를 가지고 있습니다. Transformer는 RNN 대신 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이를 통해 전체 문장을 동시에 처리하고, RNN보다 더 긴 문장을 효과적으로 처리할 수 있습니다. 또한, Transformer는 인코더와 디코더라는 두 개의 서브넷으로 구성되어 있으며, 각각 입력 문장을 인코딩하고 출력 문장을 디코딩합니다. 이러한 구조를 통해 Transformer는 번역, 요약 등의 작업에서 뛰어난 성능을 보입니다.'

In [ ]:
conversation.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='인공지능에서 Transformer가 뭐야?'), AIMessage(content='Transformer는 자연어 처리 분야에서 매우 중요한 모델입니다. 이 모델은 주로 기계 번역, 질의응답, 요약 등의 작업에 사용됩니다. Transformer는 2017년에 발표된 논문 "Attention is All You Need"에서 처음 소개되었습니다. 이 모델은 기존의 순차적인 방식이 아닌, 전체 문장을 동시에 처리하는 방식을 사용하여 성능을 크게 향상시켰습니다. Transformer는 주로 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이를 통해 문장의 의미를 잘 이해하고 번역, 요약 등의 작업을 수행할 수 있습니다.'), HumanMessage(content='RNN하고 차이 설명해줘.'), AIMessage(content='RNN은 순환 신경망(Recurrent Neural Network)의 약자로, 순차적인 데이터를 처리하는 데에 주로 사용되는 모델입니다. RNN은 이전의 입력에 의존하여 현재의 출력을 계산하는 구조를 가지고 있습니다. 이전의 정보를 기억하고 현재의 입력과 함께 사용하여 다음 출력을 예측하는 데에 유용합니다. 예를 들어, 문장의 단어를 순서대로 입력으로 받아 문장의 의미를 파악하거나, 시계열 데이터를 입력으로 받아 다음 시점의 값을 예측하는 등의 작업에 사용됩니다.\n\n반면에 Transformer는 RNN과는 다른 구조를 가지고 있습니다. Transformer는 기존의 순차적인 방식이 아닌, 전체 문장을 동시에 처리하는 방식을 사용합니다. 이를 위해 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이러한 구조를 통해 Transformer는 RNN보다 더 긴 문장을 처리하는 데에 효과적이며, 번역, 요약 등의 작

* ConversationBufferMemory : 대화 기록(기본)
* ConversationBufferWindowMemory : 마지막 n개의 대화만 기억
* Entity Memory : 개체에 대한 정보를 저장
* Conversation Knowledge Graph Memory: 개체의 triple 저장: (sam, 좋아하는 색, 파랑)
* ConversationSummaryMemory : 대화의 요약본을 저장
* ConversationSummaryBufferMemory : 대화 요약본 + 마지막 n토큰 기억
* ConversationTokenBufferMemory : 마지막 n토큰 기억
* VectorStore-Backed Memory : 벡터DB에 정보 저장

In [ ]:
#@title 7. Document Loaders
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://ko.wikipedia.org/wiki/NewJeans")

documents = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

# 4096 token = 3000 English word

23

In [ ]:
print(docs[-1].page_content)

원본 주소 "https://ko.wikipedia.org/w/index.php?title=NewJeans&oldid=35904702"
분류: K-pop 음악 그룹한국어 음악 그룹대한민국의 팝 음악 그룹대한민국의 댄스 음악 그룹대한민국의 걸 그룹5인조 음악 그룹2022년 결성된 음악 그룹2020년대 음악 그룹다국적 음악 그룹엠넷 엠카운트다운 1위 수상자KBS 뮤직뱅크 1위 수상자MBC 쇼! 음악중심 1위 수상자SBS 인기가요 1위 수상자숨은 분류: CS1 - 영국 영어 인용 (en)CS1 - 러시아어 인용 (ru)CS1 - 미국 영어 인용 (en)위키데이터 속성 P18을 사용하는 문서위키데이터 속성 P154를 사용하는 문서위키데이터 속성 P373을 사용하는 문서위키데이터 속성 P213을 사용하는 문서위키데이터 속성 P434를 사용하는 문서위키데이터 속성 P5034를 사용하는 문서베트남어 표기를 포함한 문서위키데이터와 같은 Twitter username위키데이터와 다른 Twitter usernameISNI 식별자를 포함한 위키백과 문서NLK 식별자를 포함한 위키백과 문서뮤직브레인즈 식별자를 포함한 위키백과 문서


 이 문서는 2023년 11월 11일 (토) 20:38에 마지막으로 편집되었습니다.
모든 문서는 크리에이티브 커먼즈 저작자표시-동일조건변경허락 4.0에 따라 사용할 수 있으며, 추가적인 조건이 적용될 수 있습니다. 자세한 내용은 이용 약관을 참고하십시오.Wikipedia®는 미국 및 다른 국가에 등록되어 있는 Wikimedia Foundation, Inc. 소유의 등록 상표입니다.


개인정보처리방침
위키백과 소개
면책 조항
행동 강령
개발자
통계
쿠키 정책
모바일 보기

내용 폭 제한 전환


---
* Web Page
* PDF
* Email
* Twitter
* DataFrame (pandas)
* YouTube
* Notion
* Google Drive
* Powerpoint, Word,
* ...
---

In [ ]:
#@title 8. Summarization
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(chat, chain_type="map_reduce", verbose=True)
chain.run(docs[:-1])




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"NewJeans - 위키백과, 우리 모두의 백과사전

본문으로 이동

주 메뉴

주 메뉴
사이드바로 이동
숨기기

		둘러보기
	


대문최근 바뀜요즘 화제임의의 문서로기부

		사용자 모임
	


사랑방사용자 모임관리 요청

		편집 안내
	


도움말정책과 지침질문방

언어

이 위키백과에서 언어 링크는 문서 제목 건너편의 문서 최상단에 있습니다. 최상단으로 이동합니다.

검색

검색


계정 만들기로그인


개인 도구

 계정 만들기 로그인

		로그아웃한 편집자를 위한 문서 더 알아보기

기여토론


목차
사이드바로 이동
숨기기


처음 위치

1그룹명

2역사

역사 하위섹션 토글하기

2.12011~2022: 데뷔 전 활동 및 결성

2.22022년~현재: 소개, 데뷔, 그리고 Get Up

3구성원

4음반

음반 하위섹션 토글하기

4.1EP

4.2싱글

5음반 외 활동

6수상 목록

수상 목록 하위섹션 토글하기

6.1가요 프로그램 1위

6.2시상식

7각주

8외부 링크

목차 토글

NewJeans

31개 언어


العربيةБеларускаяDeutschEnglishEsperantoEspañolEuskaraفارسیSuomiFrançaisעבריתՀայերենBahasa IndonesiaItaliano日本語Lietuviųမြန်မာဘာသာPolskiPortuguêsРусскийSlovenčinaСрпски / srpskiSvenskaไทยTürkçeУкраїнськаOʻzbekcha / ўзбекчаTiếng Việt中文Bân-lâm-gú粵語

링크 편집

문서토론

한국어

읽기편집역사 보기

도구

도구
사이드바로 이동
숨기기

		동작
	


읽기편집역

'NewJeans is a 5-member girl group from South Korea that debuted in July 2022. They are under the label "어도어" which is a subsidiary of Hive. The group was formed by producer Min Hee-jin and achieved their first number one on a music show just three weeks after their debut. They have released an EP titled "New Jeans" and have won multiple awards. They have also collaborated with brands and famous individuals in their music videos. They performed at the Lollapalooza festival in 2023 and will perform the theme song for the League of Legends World Championship.'

* "stuff": LLM 한번에 다 보냄. 길면 오류남
* "map_reduce": 나눠서 요약, 전체 요약본 다시 요약
* "refine": (요약 + 다음 문서) => 요약
* "map_rerank": 점수매겨서 중요한거로 요약

In [ ]:
#@title 9. Embeddings and VectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings()

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    # text_splitter=text_splitter,
    ).from_loaders([loader])

# 파일로 저장
index.vectorstore.save_local("faiss-nj")

* Embedding : LLM에 input(text)를 넣었을때 나오는 특징 벡터 - Neural Network의 특정 메모리를 읽어서 만들고, 보통 512개 이상의 float로 만들어짐.
* 의미상 비슷한 input이 들어갔을때, 나오는 특징 벡터가 유사한 특징을 갖고, 거리상 가깝다.
* OpenAI에서 text를 넣으면 Embedding을 돌려주는 API를 제공한다. (다만 비용 발생)
* 로컬에서 유사한 알고리즘을 돌릴수 있다. (다만 똑같은 벡터는 아니고, openAI 임베딩과는 비교할수 없다)
* HuggingFaceEmbeddings 사용하면 로컬에서 돌릴수 있고, GPU 있다면 가속도 지원함.
* Vector Database는 Embedding 벡터와 텍스트를 저장하는 DB
  * Pinecone 서비스 (유료, 무료 제한적)
  * FAISS (페이스북, 오픈소스, 로컬)
  * Embeddings -> 텍스트를 저장
  * Embeddings로 검색할수 있으며, 거리상 가장 가까운 항목을 가져올수 있음. (거리를 구하는 방법은 여러가지)
  * 유사한 의미를 갖는 텍스트에 대한 Embedding 벡터는 거리상 가깝다.

In [ ]:
index.query("뉴진스의 데뷔곡은?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스(NewJeans)의 데뷔곡은 "Attention"입니다.'

In [ ]:
index.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스(NewJeans)의 데뷔 멤버는 현재 알려진 바에 따르면 공개되지 않았습니다.'

In [ ]:
index.query("멤버의 나이는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'멤버의 나이에 대한 정보는 제공되지 않았습니다.'

In [ ]:
index.query("멤버의 나이는? (오늘은 2023년 11월 18일)", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'민지: 19세 (2004년 5월 7일 생)\n하니: 19세 (2004년 10월 6일 생)\n다니엘: 18세 (2005년 4월 11일 생)\n해린: 17세 (2006년 5월 15일 생)\n혜인: 15세 (2008년 4월 21일 생)'

In [ ]:
#@title FAISS 벡터DB 디스크에서 불러오기
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

fdb = FAISS.load_local("faiss-nj", embeddings)
index2 = VectorStoreIndexWrapper(vectorstore=fdb)

In [ ]:
index2.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스(NewJeans)의 데뷔 멤버는 현재 알려진 바에 따르면 ADOR, LEE, MIN, RAE, YUJIN, YUNA, ZOEY로 알려져 있습니다.'